In [ ]:
import dolfin
import matplotlib.pyplot as plt
import numpy as np

from bluemira.builders.plasma import MakeParameterisedPlasma
from bluemira.equilibria.fem_fixed_boundary.fem_magnetostatic_2D import (
    FemGradShafranovFixedBoundary,
)
from bluemira.equilibria.fem_fixed_boundary.utilities import plot_scalar_field
from bluemira.mesh import meshing
from bluemira.mesh.tools import import_mesh, msh_to_xdmf

# Create a plasma shape

In [ ]:
params = {
    "R_0": (9.0, "Input"),
    "A": (3.5, "Input"),
}
build_config = {
    "name": "Plasma",
    "class": "MakeParameterisedPlasma",
    "param_class": "bluemira.equilibria.shapes::JohnerLCFS",
    "variables_map": {
        "r_0": "R_0",
        "a": "A",
    },
}
builder = MakeParameterisedPlasma(params, build_config)
plasma = builder().get_component("xz").get_component("LCFS")

plasma.shape.mesh_options = {"lcar": 0.3, "physical_group": "plasma"}
plasma.shape.boundary[0].mesh_options = {"lcar": 0.3, "physical_group": "lcfs"}


Initialize and create the mesh

In [ ]:
m = meshing.Mesh()
buffer = m(plasma)


# Convert to xdmf

In [ ]:
msh_to_xdmf("Mesh.msh", dimensions=(0, 2), directory=".", verbose=True)

mesh, boundaries, subdomains, labels = import_mesh(
    "Mesh",
    directory=".",
    subdomains=True,
)
dolfin.plot(mesh)
plt.show()

Ic = 18e6

gs_solver = FemGradShafranovFixedBoundary(mesh)
gs_solver.solve(1, 0, Ic, max_iter=20, verbose_plot=True)

points = mesh.coordinates()
psi_data = np.array([gs_solver.psi(x) for x in points])

levels = np.linspace(0.0, gs_solver.psi_ax, 25)

axis, cntr, _ = plot_scalar_field(
    points[:, 0], points[:, 1], psi_data, levels=levels, axis=None, tofill=True
)
plt.show()

axis, cntr, _ = plot_scalar_field(
    points[:, 0],
    points[:, 1],
    psi_data,
    levels=[gs_solver.psi_ax * 0.05],
    axis=None,
    tofill=False,
)
plt.show()